### 시각화 코드 실행 및 테스트

- 모든 모델 시각화 관련 코드 수행
- 출력된 시각화 결과는 ./data/images에 저장 후 readme 작성할때 삽입


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
import platform

import modules.DataSelect as DataSelect
import modules.DataAnalysis as DataAnalysis
import modules.ModelAnalysis as ModelAnalysis
import modules.DataModify as DataModify
from modules.DataModify import DataPreprocessing

import modules.Models as Models


In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
### matplotlib 에서 한글 및 음수 표현이 깨지는 현상 수정

system = platform.system()

if system == "Windows":     # Windows
    rcParams['font.family'] = 'Malgun Gothic'
elif system == "Darwin":    # macOS
    rcParams['font.family'] = 'AppleGothic'

rcParams['axes.unicode_minus'] = False

In [4]:
### 데이터 로드

input_file_path1 = './data/2022Data_part1.csv'
input_file_path2 = './data/2022Data_part2.csv'
input_file_path3 = './data/Suicide2010-2021.csv'

df_part1 = pd.read_csv(input_file_path1)
df_part2 = pd.read_csv(input_file_path2)
df_part3 = pd.read_csv(input_file_path3)

df = pd.concat([df_part1, df_part2, df_part3], ignore_index=True)


---


### 데이터 분석


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537222 entries, 0 to 537221
Data columns (total 26 columns):
 #   Column                                                       Non-Null Count   Dtype 
---  ------                                                       --------------   ----- 
 0   Patient ID                                                   537222 non-null  int64 
 1   Age recode with <1 year olds and 90+                         537222 non-null  object
 2   Sex                                                          537222 non-null  object
 3   Year of diagnosis                                            537222 non-null  int64 
 4   Year of follow-up recode                                     537222 non-null  int64 
 5   Race recode (W, B, AI, API)                                  537222 non-null  object
 6   Site recode ICD-O-3/WHO 2008                                 537222 non-null  object
 7   Primary Site                                                 537222 non-nu

In [6]:
# 출력이 100줄까지는 생략되지 않도록 조정
pd.set_option('display.max_rows', 100)

DataAnalysis.show_value_counts(df, boundary=100)

Patient ID
continuous
--------------------
Age recode with <1 year olds and 90+
65-69 years    89601
70-74 years    80930
60-64 years    78022
75-79 years    60938
55-59 years    60826
50-54 years    46403
80-84 years    37299
45-49 years    24864
85-89 years    18134
40-44 years    14056
35-39 years     7819
90+ years       6976
30-34 years     4562
25-29 years     2612
20-24 years     1545
15-19 years     1099
10-14 years      556
01-04 years      512
00 years         242
05-09 years      226
Name: count, dtype: int64
--------------------
Sex
Female    269595
Male      267627
Name: count, dtype: int64
--------------------
Year of diagnosis
2022    92262
2021    67553
2020    44365
2019    41915
2018    35155
2017    30738
2016    27013
2015    23919
2014    21171
2013    18454
2012    16935
2011    15318
2010    14269
2009    13275
2008    12122
2007    11058
2006    10075
2005     8997
2004     7982
2003     7192
2002     6459
2001     5812
2000     5183
Name: count, dtype: int64
--

### 전처리 관련 모듈 안내

- `modules.DataSelect`: 생존 관련 라벨 구성을 위한 카테고리 정의와 컬럼 선택 유틸리티를 제공합니다.
- `modules.DataAnalysis`: 데이터 분포와 시각화를 지원하는 탐색용 함수를 포함합니다.
- `modules.DataModify`: 전처리 파이프라인과 데이터셋 생성을 담당하는 도구를 제공합니다.
- `modules.ModelAnalysis`: 학습된 모델의 결과를 시각화해 성능을 점검할 수 있는 함수를 담고 있습니다.
- `modules.Models`: 생존 분석 모델과 손실 함수를 정의해 학습에 활용합니다.


In [7]:
# categorical한 데이터 encoding
# category_encoding 함수는 DataSelect를 이용해 선택한 범주형 컬럼을 라벨 또는 원-핫 방식으로 수치화합니다.
def category_encoding(df, categories, encoding='label') :
    # encoding : 'label' - 라벨 인코딩, 'onehot' - One-hot 인코딩
    # categories = {'encoding_type':..., '컬럼명':{'요소':'라벨', '요소':'라벨', ...}, ...}
    # categorical한 데이터 = 데이터가 가지는 서로 다른 값이 15개 미만

    # 학습 시점마다 참조하는 범주형 컬럼명을 수집해 encoding 기준으로 활용합니다.
    categorical_col = DataSelect.return_cols(df, 'categorical', boundary=100)       # Categorical한 column의 컬럼명을 선택

    df_encoded = df.copy() # 인코딩할 데이터

    if encoding == 'label' : # 라벨 형식으로 인코딩
        categories['encoding_type'] = 'label'

        for col in categorical_col:
            unique_vals = df_encoded[col].unique()
            # 데이터에 등장한 순서대로 0, 1, 2 ... 값을 부여해 정수 라벨을 만듭니다.
            label = {val: i for i, val in enumerate(unique_vals)}
            df_encoded[col] = df_encoded[col].map(label)
            # categories 딕셔너리에 {원본값: 인코딩값} 매핑을 저장해 추후 역변환에 활용합니다.
            categories[col] = label

    elif encoding == 'onehot' : # One-hot 인코딩
        categories['encoding_type'] = 'onehot'
        
        for col in categorical_col:
            # 각 컬럼의 고유값마다 0/1 플래그를 갖는 더미 변수를 생성합니다.
            dummies = pd.get_dummies(df_encoded[col], prefix=col)
            df_encoded = pd.concat([df_encoded.drop(columns=[col]), dummies], axis=1)
            # 더미 컬럼명을 그대로 저장해 역변환 시 어떤 값이었는지 알 수 있도록 합니다.
            categories[col] = dummies.columns.tolist()

    else:   # 이상한 값이 들어오면 Value Error 발생
        raise ValueError(f"알 수 없는 encoding_type: {encoding}")

    return df_encoded   # 인코딩된 데이터를 반환

# encoding된 데이터 decoding
# category_decoding 함수는 저장해 둔 매핑 정보를 활용해 인코딩된 데이터를 원본 범주값으로 복원합니다.
def category_decoding(df = None, categories=None) :
    df_decoded = df.copy()

    if categories is not None :
        categories = categories

    encoding_type = categories.get('encoding_type', None)  # categories 데이터에 저장된 인코딩 타입에 따라 인코딩 타입 설정

    if encoding_type == 'label':    # 라벨 인코딩일 경우
        for col, mapping in categories.items():
            if col == 'encoding_type':
                continue
            # {원본값: 숫자} → {숫자: 원본값} 구조로 뒤집어서 각 컬럼을 복원합니다.
            reverse_map = {v: k for k, v in mapping.items()}
            df_decoded[col] = df_decoded[col].map(reverse_map)

    elif encoding_type == 'onehot': # One-hot 인코딩일 경우
        for col, dummy_cols in categories.items():
            if col == 'encoding_type':
                continue
            # 각 더미 컬럼 중 값이 1인 항목을 찾아 원래의 범주명을 복원합니다.
            def decode_row(row):
                for dummy_col in dummy_cols:
                    if row[dummy_col] == 1:
                        return dummy_col.replace(f"{col}_", "")
                return None
            
            df_decoded[col] = df_decoded.apply(decode_row, axis=1)
            # 복원 후 더미 컬럼은 더 이상 필요하지 않으므로 제거합니다.
            df_decoded = df_decoded.drop(columns=dummy_cols)

    else:
        raise ValueError(f"알 수 없는 encoding_type: {encoding_type}")

    return df_decoded           # dictionary 형태 categories 를 받아서, 해당 데이터를 기반으로 디코딩 후, 테이블 반환


In [8]:
categories={}

encoded = category_encoding(df, categories=categories, encoding='label')

In [9]:
display(encoded)

,Patient ID,Age recode with <1 year olds and 90+,Sex,Year of diagnosis,Year of follow-up recode,"Race recode (W, B, AI, API)",Site recode ICD-O-3/WHO 2008,Primary Site,Primary Site - labeled,Derived Summary Grade 2018 (2018+),...,COD to site recode,Sequence number,Median household income inflation adj to 2023,Number of Cores Positive Recode (2010+),Number of Cores Examined Recode (2010+),EOD Primary Tumor Recode (2018+),PRCDA 2020,Survival months flag,Survival months,Vital status recode (study cutoff used)
0,671,0,0,0,0,0,0,341,"C34.1-Upper lobe, lung",0,...,0,0,0,0,0,0,0,0,0059,0
1,812,0,1,1,0,0,1,163,C16.3-Gastric antrum,1,...,0,1,0,0,0,1,0,0,0007,0
2,878,1,0,2,0,0,2,181,C18.1-Appendix,2,...,0,0,1,0,0,2,0,0,0218,0
3,1018,2,1,3,0,0,3,182,C18.2-Ascending colon,2,...,0,2,2,0,0,2,0,0,0134,0
4,1269,3,1,4,0,0,4,187,C18.7-Sigmoid colon,2,...,0,1,2,0,0,2,0,0,0187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537217,63269474,8,0,7,11,0,36,649,"C64.9-Kidney, NOS",3,...,38,1,12,0,0,1,0,3,0000,1
537218,63270712,4,0,7,11,0,0,343,"C34.3-Lower lobe, lung",3,...,38,1,11,0,0,11,0,0,0000,1
537219,63307996,2,0,5,3,1,71,140,"C14.0-Pharynx, NOS",3,...,38,1,4,0,0,3,0,2,Unknown,1
537220,63329167,1,0,7,11,0,10,180,C18.0-Cecum,4,...,38,1,13,0,0,3,0,0,0001,1


In [10]:
print(categories)

{'encoding_type': 'label', 'Age recode with <1 year olds and 90+': {'65-69 years': 0, '60-64 years': 1, '70-74 years': 2, '40-44 years': 3, '45-49 years': 4, '85-89 years': 5, '55-59 years': 6, '75-79 years': 7, '30-34 years': 8, '80-84 years': 9, '50-54 years': 10, '90+ years': 11, '35-39 years': 12, '20-24 years': 13, '01-04 years': 14, '10-14 years': 15, '15-19 years': 16, '25-29 years': 17, '00 years': 18, '05-09 years': 19}, 'Sex': {'Male': 0, 'Female': 1}, 'Year of diagnosis': {np.int64(2018): 0, np.int64(2022): 1, np.int64(2004): 2, np.int64(2011): 3, np.int64(2007): 4, np.int64(2020): 5, np.int64(2019): 6, np.int64(2021): 7, np.int64(2016): 8, np.int64(2010): 9, np.int64(2015): 10, np.int64(2003): 11, np.int64(2012): 12, np.int64(2005): 13, np.int64(2013): 14, np.int64(2017): 15, np.int64(2002): 16, np.int64(2009): 17, np.int64(2006): 18, np.int64(2014): 19, np.int64(2000): 20, np.int64(2001): 21, np.int64(2008): 22}, 'Year of follow-up recode': {np.int64(2022): 0, np.int64(201

---


In [11]:
### encoding
from typing import Dict, Iterable, List, Optional, Tuple

# 나이 그룹
AGE_RECODE_ORDER: List[str] = [
    '00 years',
    '01-04 years',
    '05-09 years',
    '10-14 years',
    '15-19 years',
    '20-24 years',
    '25-29 years',
    '30-34 years',
    '35-39 years',
    '40-44 years',
    '45-49 years',
    '50-54 years',
    '55-59 years',
    '60-64 years',
    '65-69 years',
    '70-74 years',
    '75-79 years',
    '80-84 years',
    '85-89 years',
    '90+ years'
]

# 나이, 진단 일자, 관측 일자를 순서형으로 처리하기 위한 기본 설정
ORDINAL_CONFIG_DEFAULT: Dict[str, Iterable] = {
    'Age recode with <1 year olds and 90+': AGE_RECODE_ORDER,
    'Year of diagnosis': 'numeric',
    'Year of follow-up recode': 'numeric',
}


# _build_group_map 함수는 사인·생존 정보 그룹 정의를 인덱스 매핑 딕셔너리로 변환합니다.
def _build_group_map(definitions: Iterable[Iterable[str]]) -> Dict[str, int]:
    mapping: Dict[str, int] = {}
    for idx, group in enumerate(definitions):
        for value in group:
            mapping[value] = idx
    return mapping


# DataSelect 모듈에 정의된 생존 플래그 / 사망 원인 그룹을 인덱스로 치환합니다.
SURVIVAL_FLAG_GROUP_MAP = _build_group_map(DataSelect.label_Surv_flags)
COD_GROUP_MAP = _build_group_map(DataSelect.label_cod_list)
# COD_GROUP_TO_TARGET: 위 두 매핑을 결합해 최종 타깃 레이블(사건 유형)을 결정합니다.
#   0번 그룹 → 암 관련 사망, 1번 → 합병증 사망, 2번 → 기타 질병 사망, 3번 → 생존/외인사(-1), 4번 → 자살(3)
COD_GROUP_TO_TARGET = {0: 0, 1: 1, 2: 2, 3: -1, 4: 3}
TARGET_DESCRIPTION = {
    -1: 'Alive or external cause',
    0: 'Cancer-related death',
    1: 'Complication-related death',
    2: 'Other disease-related death',
    3: 'Suicide or self-inflicted',
}


# bin_survival_months 함수는 생존 개월 수를 지정된 구간 크기(기본 3개월)로 나눠 이산형 지표를 생성합니다.
def bin_survival_months(series: pd.Series, bin_size: int = 3) -> pd.Series:
    if bin_size <= 0:
        raise ValueError('bin_size must be a positive integer')
    # 문자열로 들어온 개월 수를 안전하게 숫자로 변환하고, 변환 실패값은 결측으로 처리합니다.
    numeric = pd.to_numeric(series, errors='coerce')

    # 모델 안정성을 위해 270개월(=90개 3개월 구간) 이상은 270으로 상한을 둡니다.
    numeric = numeric.clip(upper=270)
    # 정수 나눗셈을 통해 0,1,2,... 형태의 구간 인덱스를 만들고 Int64(nullable) 타입으로 유지합니다.
    binned = (numeric // bin_size).astype('Int64')
    # 원본이 결측이면 그대로 NA를 유지하도록 보정합니다.
    binned = binned.where(~numeric.isna(), other=pd.NA)
    # 결측을 -1로 치환해 모델 입력 시 구간을 놓치지 않도록 합니다.
    return binned.fillna(-1).astype(int)


# encode_ordinal_columns 함수는 사전 정의한 순서를 참고해 순서형 변수를 정수 라벨로 변환합니다.
def encode_ordinal_columns(
    df: pd.DataFrame,
    ordinal_config: Optional[Dict[str, Iterable]] = None
) -> Tuple[pd.DataFrame, Dict[str, Dict[str, int]]]:
    config = ordinal_config or ORDINAL_CONFIG_DEFAULT
    df_encoded = df.copy()
    mappings: Dict[str, Dict[str, int]] = {}

    for col, definition in config.items():
        if col not in df_encoded.columns:
            continue
        series = df_encoded[col]
        present_values = series.dropna().unique()
        if isinstance(definition, str):
            if definition != 'numeric':
                raise ValueError(f"Unsupported ordinal definition: {definition}")
            # 숫자형 오름차순 정렬을 보장하기 위해 먼저 숫자 변환을 시도합니다.
            numeric_series = pd.to_numeric(pd.Series(present_values), errors='coerce')
            if numeric_series.isna().all():
                # 모두 숫자로 변환되지 않으면 문자열 정렬로 fallback 합니다.
                order = sorted(present_values, key=lambda x: str(x))
            else:
                # 숫자로 변환된 값은 (변환값, 원본값) 쌍으로 묶어 낮은 값부터 정렬합니다.
                ordered_pairs = sorted(
                    [(num, val) for num, val in zip(numeric_series, present_values) if not pd.isna(num)],
                    key=lambda pair: pair[0]
                )
                order = [val for _, val in ordered_pairs]
                # 숫자로 환산되지 않는 값은 뒤쪽에 문자열 기준으로 정렬해 붙입니다.
                non_numeric_vals = [val for num, val in zip(numeric_series, present_values) if pd.isna(num)]
                order.extend(sorted(non_numeric_vals, key=lambda x: str(x)))
        else:
            # definition이 리스트라면 지정된 순서를 최대한 유지하고, 정의 밖의 값은 사전식으로 뒤에 정렬합니다.
            present_set = set(present_values)
            order = [value for value in definition if value in present_set]
            remaining = sorted(present_set - set(order), key=str)
            order.extend(remaining)
        # 최종적으로 {범주값: 순위} 매핑을 만들고 결측은 -1로 통일합니다.
        mapping = {value: idx for idx, value in enumerate(order)}
        mapping['__MISSING__'] = -1
        df_encoded[col] = series.map(mapping).fillna(-1).astype(int)
        mappings[col] = mapping

    return df_encoded, mappings


# encode_nominal_columns 함수는 순서가 없는 범주형 컬럼을 팩터라이즈하여 모델 입력용 정수 라벨로 만듭니다.
def encode_nominal_columns(
    df: pd.DataFrame,
    exclude_columns: Optional[Iterable[str]] = None
) -> Tuple[pd.DataFrame, Dict[str, Dict[str, int]]]:
    excludes = set(exclude_columns or [])
    df_encoded = df.copy()
    mappings: Dict[str, Dict[str, int]] = {}

    for col in df_encoded.columns:
        if col in excludes or pd.api.types.is_numeric_dtype(df_encoded[col]):
            continue
        # factorize는 등장 순서를 기준으로 정수 코드를 배정합니다.
        series = df_encoded[col].astype('object')
        codes, uniques = pd.factorize(series, sort=True)
        df_encoded[col] = codes
        mapping = {value: idx for idx, value in enumerate(uniques)}
        mapping['__MISSING__'] = -1
        mappings[col] = mapping

    return df_encoded, mappings


# create_combined_label 함수는 사망 원인과 생존 상태 컬럼을 종합해 다중 클래스 타깃 레이블을 만듭니다.
def create_combined_label(
    df: pd.DataFrame,
    cod_col: str = 'COD to site recode',
    survival_flag_col: str = 'Survival months flag',
    vital_status_col: str = 'Vital status recode (study cutoff used)'
) -> Tuple[pd.Series, pd.Series, Dict[int, str]]:
    if survival_flag_col not in df or vital_status_col not in df or cod_col not in df:
        missing = [col for col in [cod_col, survival_flag_col, vital_status_col] if col not in df]
        raise KeyError(f"Missing required columns: {missing}")

    # 관측 상태 플래그를 그룹 인덱스로 치환해 사용 불가한 표본을 골라내기 위한 기초 데이터로 사용합니다.
    survival_groups = df[survival_flag_col].map(SURVIVAL_FLAG_GROUP_MAP)
    vital_status = df[vital_status_col]

    # 1) 생존 플래그가 정의되지 않은 경우
    # 2) 조직상으로 생존 기간이 0개월로 잘못 표기된 채 사망 처리된 경우
    # 3) 데이터셋에서 제외해야 하는 특수 플래그(예: 추적 불가)인 경우를 제거합니다.
    drop_mask = (
        survival_groups.isna()
        | (survival_groups.eq(2) & vital_status.eq('Dead'))
        | survival_groups.eq(3)
    )
    valid_mask = ~drop_mask

    labels = pd.Series(pd.NA, index=df.index, dtype='Int64')
    # 생존으로 판단되는 표본은 -1(Alive) 클래스로 부여합니다.
    alive_mask = valid_mask & vital_status.eq('Alive')
    labels.loc[alive_mask] = -1

    # 사망 표본은 사망 원인 그룹을 통해 COD_GROUP_TO_TARGET 규칙에 따라 재분류합니다.
    dead_mask = valid_mask & vital_status.eq('Dead')
    if dead_mask.any():
        cod_groups = df.loc[dead_mask, cod_col].map(COD_GROUP_MAP)
        labels.loc[dead_mask] = cod_groups.map(COD_GROUP_TO_TARGET)

    # 유효한 타깃만 남기고 Int 형으로 변환합니다.
    final_mask = labels.notna()
    labels = labels.loc[final_mask].astype(int)
    labels.name = 'target_label'
    return labels, final_mask, TARGET_DESCRIPTION


# preprocess_for_model 함수는 전체 데이터를 복사한 뒤 타깃 생성, 인코딩, 메타 정보 구성을 한 번에 수행합니다.
def preprocess_for_model(
    df: pd.DataFrame,
    ordinal_config: Optional[Dict[str, Iterable]] = None,
    bin_size: int = 3,
    survival_months_col: str = 'Survival months',
    cod_col: str = 'COD to site recode',
    survival_flag_col: str = 'Survival months flag',
    vital_status_col: str = 'Vital status recode (study cutoff used)',
    drop_label_source: bool = True
) -> Tuple[pd.DataFrame, pd.Series, Dict[str, Dict]]:
    df_work = df.copy()

    # 1. 사망 원인/생존 정보로부터 학습에 사용할 타깃 라벨과 유지할 인덱스를 생성합니다.
    labels, valid_mask, label_desc = create_combined_label(
        df_work,
        cod_col=cod_col,
        survival_flag_col=survival_flag_col,
        vital_status_col=vital_status_col,
    )
    df_work = df_work.loc[labels.index].copy()
    df_work['target_label'] = labels.astype(int)

    # 2. 생존 개월 수가 있으면 일정 간격으로 구간화한 파생 컬럼을 추가해 모델 입력 다양성을 확보합니다.
    bin_col_name = None
    if survival_months_col in df_work.columns:
        df_work[survival_months_col] = pd.to_numeric(df_work[survival_months_col], errors='coerce')
        # 구간화와 일관성을 위해 원본 생존 개월도 270개월로 상한을 둡니다.
        df_work[survival_months_col] = df_work[survival_months_col].clip(upper=270)
        bin_col_name = f'{survival_months_col}_bin_{bin_size}m'
        df_work[bin_col_name] = bin_survival_months(df_work[survival_months_col], bin_size=bin_size)

    # 3. 순서형 컬럼(나이, 연도 등)을 지정된 규칙에 따라 정수 라벨로 변환합니다.
    df_work, ordinal_mappings = encode_ordinal_columns(df_work, ordinal_config)

    # 4. 타깃 컬럼과 순서형 컬럼은 명목형 인코딩 대상에서 제외합니다.
    exclude = set((ordinal_config or ORDINAL_CONFIG_DEFAULT).keys())
    exclude.add('target_label')
    if survival_months_col in df_work.columns:
        exclude.add(survival_months_col)
    if bin_col_name:
        exclude.add(bin_col_name)
    if drop_label_source:
        # 타깃을 만든 원천 컬럼(COD, 생존 플래그 등)은 모델 입력에서 제거합니다.
        for col in [cod_col, survival_flag_col, vital_status_col]:
            if col in df_work.columns:
                df_work = df_work.drop(columns=col)
    else:
        exclude.update([cod_col, survival_flag_col, vital_status_col])

    # 5. 남은 범주형 컬럼을 명목형 인코딩으로 수치화합니다.
    df_work, nominal_mappings = encode_nominal_columns(df_work, exclude_columns=exclude)
    df_work = df_work.reset_index(drop=True)

    # 6. 학습 및 추후 역변환/분석에 필요한 메타 정보를 묶어 반환합니다.
    meta = {
        'ordinal_mappings': ordinal_mappings,
        'nominal_mappings': nominal_mappings,
        'label_description': label_desc,
        'label_column': 'target_label',
        'survival_bin_column': bin_col_name,
        'bin_size_months': bin_size,
        'retained_mask': valid_mask,
        'retained_index': labels.index,
    }

    return df_work, df_work['target_label'], meta


In [12]:
dp = DataPreprocessing()
df_preprocessed, target_series, preprocess_meta = dp.run(df, mode='full')
display(df_preprocessed.head())
print(target_series.value_counts())
print(df_preprocessed['Combined Summary Stage with Expanded Regional Codes (2004+)'].unique())

df_preprocessed.to_csv('data/encoded_dataset.csv')

,Patient ID,Age recode with <1 year olds and 90+,Sex,Year of diagnosis,Year of follow-up recode,"Race recode (W, B, AI, API)",Site recode ICD-O-3/WHO 2008,Primary Site,Primary Site - labeled,Derived Summary Grade 2018 (2018+),...,RX Summ--Surg Oth Reg/Dis (2003+),Sequence number,Median household income inflation adj to 2023,Number of Cores Positive Recode (2010+),Number of Cores Examined Recode (2010+),EOD Primary Tumor Recode (2018+),PRCDA 2020,Survival months,target_label,Survival months_bin_3m
0,671,14,0,2018,2022,0,0,341,0,0,...,0,0,0,0,0,0,0,59,-1,19
1,812,14,1,2022,2022,0,1,163,1,1,...,0,1,0,0,0,1,0,7,-1,2
2,878,13,0,2004,2022,0,2,181,2,2,...,0,0,1,0,0,2,0,218,-1,72
3,1018,15,1,2011,2022,0,3,182,3,2,...,0,2,2,0,0,2,0,134,-1,44
4,1269,9,1,2007,2022,0,4,187,4,2,...,0,1,2,0,0,2,0,187,-1,62


target_label
-1    458125
 0     52496
 3      8839
 2      7676
 1      7287
Name: count, dtype: int64
[2 1 3 9 0]


### 모델 분석
